In [2]:
# FAISS 사용
import faiss
import numpy as np

In [3]:
# 데이터 생성
np.random.seed(42)
d = 128                           # 벡터 차원
n =  10000                        # 벡터 개수
xb = np.random.random((n, d)).astype('float32')
xq = np.random.random((5, d)).astype('float32')  # 쿼리 벡터 5개

# 정확한 검색
index_flat = faiss.IndexFlatL2(d)  # L2 거리 사용
index_flat.add(xb)                 # 벡터 추가

# 상위 4개 검색
k = 4
D, I = index_flat.search(xq, k)   # 쿼리당 상위 4개 검색
print('Flat index results:')
print(f'거리:{D}')
print(f'인덱스:{I}')

Flat index results:
거리:[[13.34696   14.548449  14.708295  14.756075 ]
 [12.774043  13.365513  13.566153  13.625467 ]
 [14.6441    15.4164095 15.69348   15.726439 ]
 [12.672463  13.022816  13.477383  13.641588 ]
 [13.678085  13.78643   13.91506   14.109114 ]]
인덱스:[[8769 9385   82 5125]
 [3314 7078 2779 2931]
 [3439 3014 4097 6304]
 [8808 9253 5930 7506]
 [7086 7630   59 6994]]


IVF (inverted file index)

원리 : 벡터 공간을 여러 클러스터로 분할하고, 검색 시 관련 클러스터만 탐색

In [4]:
d = 128
n = 10000
xb = np.random.random((n, d)).astype('float32')
# IVF (inverted file index)
nlist = 100  # 클러스터 개수
quantizer = faiss.IndexFlatL2(d)  # 클러스터링을 위한 양자화기
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
index_ivf.train(xb)               # 클러스터링 학습
index_ivf.add(xb)                 # 벡터 추가
index_ivf.nprobe = 10              # 검색 시 탐색할 클러스터 개수
D, I = index_ivf.search(xq, k)   # 쿼리당 상위 4개 검색
print('IVF index results:')
print(f'거리:{D}')
print(f'인덱스:{I}')

IVF index results:
거리:[[14.55662   15.072665  15.092655  15.2474575]
 [13.335398  13.420279  13.639841  13.941099 ]
 [13.793321  15.326709  15.663305  16.24126  ]
 [13.186863  13.424326  14.151279  14.3696995]
 [14.639553  14.832081  14.97168   15.09112  ]]
인덱스:[[1756 1350  164 1258]
 [3798 4815 1313 5029]
 [9972 5979  586 2202]
 [1638 6651 9522 6693]
 [7224 2100 5022 5234]]


# Product Quantization (PQ)
원리 : 고차원 벡터를 여러 서브벡터로 나누고 각각을 양자화

In [6]:

d = 128
n = 10000
xb = np.random.random((n, d)).astype('float32')
# Product Quantization (PQ)
m = 16  # 서브벡터 개수
nlist = 100  # 클러스터 개수
nbits = 8  # 양자화 비트 수
quantizer = faiss.IndexFlatL2(d)
index_pq = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)
index_pq.train(xb)                # 클러스터링 및 양자화 학습
index_pq.add(xb)                  # 벡터 추가
index_pq.nprobe = 10               # 검색 시 탐색할 클러스터 개
D, I = index_pq.search(xq, k)    # 쿼리당 상위 4개 검색
print('PQ index results:')
print(f'거리:{D}')
print(f'인덱스:{I}')

PQ index results:
거리:[[13.4167185 13.579676  13.670931  13.835444 ]
 [11.402033  12.175816  12.422449  12.799644 ]
 [13.598275  13.91415   13.974592  14.053891 ]
 [12.184086  12.382458  12.821776  12.882677 ]
 [13.066223  13.127146  13.704638  13.827762 ]]
인덱스:[[ 523 2363 2142 2680]
 [3964 8629 2858 4650]
 [5262  694 4880 1111]
 [6572 7259 8365 2358]
 [5170 6034 2326 1683]]


HNSW (Hierarchical Navigable Small World)
원리 : 소규모 그래프 구조를 사용해 근접 이웃 탐색을 효율적으로 수행

In [7]:
d = 128
n = 10000
xb = np.random.random((n, d)).astype('float32')
# HNSW (Hierarchical Navigable Small World)
M = 32  # 그래프 연결 수
index = faiss.IndexHNSWFlat(d, M)
# 인덱스 구축시 탐색 깊이
index.hnsw.efConstruction = 40
index.add(xb)                   # 벡터 추가
# 검색시 탐색 깊이
index.hnsw.efSearch = 16
D, I = index.search(xq, k)    # 쿼리당 상위 4개 검색
print('HNSW index results:')
print(f'거리:{D}')
print(f'인덱스:{I}')


HNSW index results:
거리:[[14.783354  14.94495   15.072698  15.118013 ]
 [13.135767  13.329927  13.46833   13.856585 ]
 [15.097163  15.177838  15.525999  15.662046 ]
 [13.302759  13.874646  14.343724  14.392704 ]
 [14.2860775 14.358206  14.86422   14.992182 ]]
인덱스:[[7986 6323 3534 5689]
 [4935  158 7039 2125]
 [3713 3346 3815  643]
 [5348 6723 8647 9401]
 [7470 2556 8588 8441]]
